In [ ]:
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
sc = SparkContext(appName='Cordirant locating')

In [ ]:
ssc = StreamingContext(sc,10)

In [ ]:
ssc.checkpoint("Checkpoint quadrant")

In [ ]:
def get_quadrant(line):
    # Convert the input string into a pair of numbers
    try:
        (x, y) = [float(x) for x in line.split()]
    except:
        print ("Invalid input")
        return ('Invalid points', 1)

    # Map the pair of numbers to the right quadrant
    if x > 0 and y > 0:
        quadrant = 'First quadrant'
    elif x < 0 and y > 0:
        quadrant = 'Second quadrant'
    elif x < 0 and y < 0:
        quadrant = 'Third quadrant'
    elif x > 0 and y < 0:
        quadrant = 'Fourth quadrant'
    elif x == 0 and y != 0:
        quadrant = 'Lies on Y axis'
    elif x != 0 and y == 0:
        quadrant = 'Lies on X axis'
    else:
        quadrant = 'Origin'

    # The pair represents the quadrant and the counter increment
    return (quadrant, 1)

In [ ]:
lines = ssc.socketTextStream('20.0.41.53',6781)

In [ ]:
updateFunction = lambda new_values, running_count: sum(new_values) + (running_count or 0)

In [ ]:
running_counts = lines.map(get_quadrant).updateStateByKey(updateFunction)

In [ ]:
running_counts.pprint()

In [ ]:
ssc.start()

In [ ]:
ssc.awaitTermination()